Assignment 4

In [3]:
import random as rn
import numpy as np
import time as tm

Function for creating a random matrix. For the experiments seed 1519026460 was used.

In [4]:
def get_random_matrix(n):
    a=np.zeros((n,n),float)
    SEED = int(tm.time())
    rn.seed(SEED)
    print('seed:' ,SEED)
    for i in range(n):
        for j in range(n):
            a[i,j]=rn.random()
    return a

Functions for LU decomposition with pivoting of a matrix

In [5]:
def lu_decomposition(a,n):
    for j in range(0,n):
        make_pivot(a,j,n)
        for i in range(j+1,n):
            multipl=a[i,j]/a[j,j]
            for k in range(j,n):
                a[i,k]-=a[j,k]*multipl
            a[i,j]=multipl
    return


def make_pivot(a,prime,n):
    var=prime
    max_val=abs(a[prime,prime])
    temp_arr=np.zeros(n,float)
    for i in range(prime+1,n):
        temp=abs(a[i,prime])
        if temp>max_val:
            var=i
            max_val=temp
    if(prime!=var):
            for i in range(0,n):
                temp_arr[i]=a[prime,i]
                a[prime,i]=a[var,i]
                a[var,i]=temp_arr[i]
    return

Functions for forward and backward processes

In [6]:
def forward(a,b,n):
    d=np.zeros(n,float)
    for i in range(0,n):
        total=0.0
        for j in range(0,i):
            total=total+a[i,j]*d[j]
        d[i]=b[i]-total
    return d

def backward(a,b,n):
    d=forward(a,b,n)
    x=np.zeros(n,float)
    i=n-1
    while (i>=0):
        total=0.0
        for j in range(i+1,n):
            total=total+a[i,j]*x[j]
        x[i]=(d[i]-total)/a[i,i]
        i=i-1
    return x



Calculating inverse

In [7]:
def get_inverse(a,n):
    lu_decomposition(a,n)
    a_inverse=np.zeros((n,n),float)
    for i in range(0,n):
        b=np.zeros(n,float)
        b[i]=1.0
        a_inverse[i]=backward(a,b,n)
    return a_inverse.transpose()

To analyze the accuracy of calculations, we need to multiply $\ A$ by $\ A^{-1}$ , for that reason the following functions are important

In [8]:
def get_mult(x,y,n):
    z=np.zeros((n,n),float)
    for i in range(0,n):
        total=0.0
        for j in range(0,n):
            total=0.0
            for k in range(0,n):
                total+=(x[i,k]*y[k,j])
            z[i,j]=total
    return z



In [9]:
def get_norm(a,n):
    total=0.0
    for i in range(0,n):
        for j in range(0,n):
            total+=a[i,j]**2
    return np.sqrt(total)

Creating a matrix and checking the accuracy of $\ A*A^{-1}$ 

In [10]:
n=20
a=get_random_matrix(n)
inv=get_inverse(a,n)
a=get_random_matrix(n)
result=get_norm(get_mult(a,inv,n),n)
print('A*A^(-1) normalization: ',result)

seed: 1519026719
seed: 1519026719
A*A^(-1) normalization:  4.472135955


The result for the normalization of this multiplication should be 1, according to Linear Algebra, although the experimental value is 4.472135955. Thus, the analysis of condition number is needed.

Basically, condition number shows us the dependance between the output error and the input error. 
By the formula (Chapra & Canale 290)
$$\frac{||\Delta X||}{||X||}<=Cond[A]\frac{||\Delta A||}{||A||}$$
We can rewrite it using $\ A^{-1}$ instead of $\ X $
$$\frac{||\Delta A^{-1}||}{||A^{-1}||}<=Cond[A]\frac{||\Delta A||}{||A||}$$
Thus,
$$||\Delta A^{-1}||<=Cond[A]\frac{||\Delta A||}{||A||}||A^{-1}||$$

In [11]:
cond_number=get_norm(a,n)*get_norm(inv,n)
print('cond_number of A: ',cond_number)

cond_number of A:  223.441656941


The condition number is 207.675344945. This shows that the system is ill conditioned. In ideal situation with precise inverse matrix calculation the condition number should be equal to 1. 

To estimate the accuracy of the $\ A^{-1}$ the following formula is used.
$$||\Delta A^{-1}||<=Cond[A]\frac{||\Delta A||}{||A||}||A^{-1}||$$ 
As the limitations of the computer force $\Delta A=10^{-16}$ and the condition number is >  $\ 10^{2}$,
we can anticipate the error of the inverse matrix to be in power -14

In [12]:
error=cond_number*(1e-16)*get_norm(inv,n)/get_norm(a,n)
print(error)

4.05218839511e-14


The error is 2.96803030267e-14. It is 100 times larger than $\Delta A$, this the system can be called ill-conditioned

In [ ]:
%timeit -n 1000 a1=get_inverse(a,20)

In [18]:
%timeit -n 1000 a2=np.linalg.inv(a)

36.6 µs ± 9.29 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


The function from the linalg library is 300 times faster than my system. The difference might be in more sophisticated matrix storage, less loops and usage of FORTRAN functions for numerical calculations. 